# Trabalho de Tópicos Especiais II (unisinos 2019/02)
## Uso de processamento de linguagem natural para identificação de análises positivas e negativas, através de reviews em hoteis da europa.

### Trabalho realizado por:
- Francielle
- Gabriel Castro
- Matheus Hans
- Savana

In [2]:
import numpy # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk # natural language toolkit
import matplotlib
import matplotlib.pyplot as pyplot # use to plot graphs like MATLAB

from scipy import linalg # using to processing matrix objects
from nltk.corpus import treebank
from tkinter import *
from IPython.display import Image

matplotlib.use('Agg')

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('treebank')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/Repositórios/GitHub/Personal/Unisinos-TopicosEspeciaisII-PLN-LeituraTextual'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabriel.fernandes\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gabriel.fernandes\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\gabriel.fernandes\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\gabriel.fernandes\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\gabriel.fernandes\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package treebank is already up-to-date!


True

In [3]:
# Reading the dataset.
data = pandas.read_csv(r'src/review_hotel.csv')

In [4]:
# Exploring the top 5 observations.
data.head()

,Review
0,I am so angry that i made this post available...
1,Rooms are nice but for elderly a bit difficul...
2,My room was dirty and I was afraid to walk ba...
3,You When I booked with your company on line y...
4,Backyard of the hotel is total mess shouldn t...


In [7]:
# Exploring the last 5 observations.
data.tail()

,Review
338,Bed was really comfortable
339,Beautiful hotel with views of the park Transp...
340,No Positive
341,Large clean rooms with a nice size comfy bed ...
342,Well equipped nice room Traditinal but modern...


In [4]:
# describing the datasets.
# data.describe()

# null or missing values in the dataset.
# data.isnull().sum()

# checking the assumption -> 0 reviews will have missing values in last_review and reviews_per_month columns.
# assumption_test = data.loc[(data.last_review.isnull()) & (data.reviews_per_month.isnull()), ['number_of_reviews',  'last_reviews', 'reviews_per_month']]
# assumption_test.head()

# check the shape of the created dataframe and the number of null values
# assumption_test.shape

# filling the missing values in reviews_per_month with 0.
# data.reviews_per_month.fillna(0, inplace=True)

# Checking if the changes made are reflected.
# data.isnull().sum()

## Exemple of an information extraction architecture 

<img src="src/img/ie-architecture-extraction-information.png" height="600" width="600">

In [9]:
# get one row from data frame and convert to string 
# after, get each word on the phrase

# it's possible use nltk.tokenize.sent_tokenize(text) to split phrases, i.e. A beautifull day. I'll workout. ==> ['A beautifull day.', 'I'll workout.]
# the only problem here is that the function return the dot together, so, maybe after this use a replace function would be necessary

sentence = data.iloc[1]['Review']
tokens = nltk.word_tokenize(sentence)
tokens

['Rooms',
 'are',
 'nice',
 'but',
 'for',
 'elderly',
 'a',
 'bit',
 'difficult',
 'as',
 'most',
 'rooms',
 'are',
 'two',
 'story',
 'with',
 'narrow',
 'steps',
 'So',
 'ask',
 'for',
 'single',
 'level',
 'Inside',
 'the',
 'rooms',
 'are',
 'very',
 'very',
 'basic',
 'just',
 'tea',
 'coffee',
 'and',
 'boiler',
 'and',
 'no',
 'bar',
 'empty',
 'fridge']

### Link to understand what means each part-of-speech tag and chunk tags

- <a href="https://www.clips.uantwerpen.be/pages/mbsp-tags">Penn Treebank II tag set</a>

In [11]:
tagged = nltk.pos_tag(tokens)
tagged

[('Rooms', 'NNS'),
 ('are', 'VBP'),
 ('nice', 'JJ'),
 ('but', 'CC'),
 ('for', 'IN'),
 ('elderly', 'RB'),
 ('a', 'DT'),
 ('bit', 'NN'),
 ('difficult', 'JJ'),
 ('as', 'IN'),
 ('most', 'JJS'),
 ('rooms', 'NNS'),
 ('are', 'VBP'),
 ('two', 'CD'),
 ('story', 'NN'),
 ('with', 'IN'),
 ('narrow', 'JJ'),
 ('steps', 'NNS'),
 ('So', 'RB'),
 ('ask', 'VB'),
 ('for', 'IN'),
 ('single', 'JJ'),
 ('level', 'NN'),
 ('Inside', 'IN'),
 ('the', 'DT'),
 ('rooms', 'NNS'),
 ('are', 'VBP'),
 ('very', 'RB'),
 ('very', 'RB'),
 ('basic', 'JJ'),
 ('just', 'RB'),
 ('tea', 'JJ'),
 ('coffee', 'NN'),
 ('and', 'CC'),
 ('boiler', 'NN'),
 ('and', 'CC'),
 ('no', 'DT'),
 ('bar', 'NN'),
 ('empty', 'RB'),
 ('fridge', 'NN')]

In [12]:
# identify named entities, if it's a person or something different, like a organization
entities = nltk.chunk.ne_chunk(tagged)

In [13]:
print(entities)

(S
  Rooms/NNS
  are/VBP
  nice/JJ
  but/CC
  for/IN
  elderly/RB
  a/DT
  bit/NN
  difficult/JJ
  as/IN
  most/JJS
  rooms/NNS
  are/VBP
  two/CD
  story/NN
  with/IN
  narrow/JJ
  steps/NNS
  So/RB
  ask/VB
  for/IN
  single/JJ
  level/NN
  Inside/IN
  the/DT
  rooms/NNS
  are/VBP
  very/RB
  very/RB
  basic/JJ
  just/RB
  tea/JJ
  coffee/NN
  and/CC
  boiler/NN
  and/CC
  no/DT
  bar/NN
  empty/RB
  fridge/NN)


In [14]:
#t = treebank.parsed_sents()[entities]

TypeError: 'int' object is not iterable

In [ ]:
#t.draw()

In [ ]:
grammar = nltk.CFG.fromstring("""
    S -> NP VP
    PP -> P NP
    NP -> Det N | Det N PP | 'I'
    VP -> V NP | VP PP
    Det -> 'in' | 'my'
    N -> 'game' | 'sport'
    V -> 'doing'
    P -> 'in'
    """)

In [ ]:
parser = nltk.ChartParser(grammar)
for tree in parser.parser(tokens):
    print(tree)